In [54]:
import os
import re
from tqdm.notebook import tqdm
tqdm.pandas()
#all_files=[i for i in os.listdir()if i.endswith('xlsx')]
#print(all_files)
import pandas as pd
import numpy as np
df = pd.read_excel('TrainingDataSet_Integer.xlsx', index_col=None, header=0,dtype={'item_name': str, 'description': str,'establishment_type': str, 'CAT_Name':str},usecols=['item_name','description','establishment_type','CAT_Name'])
#df = pd.read_excel('TrainingDataSet_Integer.xlsx', index_col=None, header=0,usecols=['restaurant_UUID','item_uuid','item_name','description','establishment_type','merchant_type_analytics','CAT_Name'])


In [55]:
df.head()

,item_name,description,establishment_type,CAT_Name
0,\t\t Kendall Jackson Sauvignon Blanc,the,GROCERY,CAT_WINE
1,"\t Bogle Phantom Red Wine, Proprietary Red",the,GROCERY,CAT_WINE
2,\t Deloach Heritage Merlot 750 ML,the,GROCERY,CAT_WINE
3,\t Kendall Jackson Pinot Gris,the,GROCERY,CAT_WINE
4,\t Kendall-Jackson Pinot Noir,the,GROCERY,CAT_WINE


In [5]:
df["description"].fillna("the", inplace = True)

In [5]:
df["description"].fillna("the", inplace = True)

In [6]:
df1=df.drop_duplicates(subset=['item_name','description','establishment_type','CAT_Name'],ignore_index=True)

In [7]:
df1.shape

(167452, 4)

In [8]:
df2= df1.groupby(['item_name','description','establishment_type'])['CAT_Name'].apply(','.join).reset_index()

In [9]:
df2.shape

(159707, 4)

In [10]:
df2.head()

,item_name,description,establishment_type,CAT_Name
0,\t\t Kendall Jackson Sauvignon Blanc,the,GROCERY,CAT_WINE
1,"\t Bogle Phantom Red Wine, Proprietary Red",the,GROCERY,CAT_WINE
2,\t Deloach Heritage Merlot 750 ML,the,GROCERY,CAT_WINE
3,\t Kendall Jackson Pinot Gris,the,GROCERY,CAT_WINE
4,\t Kendall-Jackson Pinot Noir,the,GROCERY,CAT_WINE


In [11]:
#df2['input_str']=df2['item_name']+df2['description']+df2['establishment_type']
df2['input_str'] = df2[['item_name', 'description', 'establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)

In [12]:
df3=df2.copy()

In [13]:
import re
import string
def preprocess(text):
    text = text.lower()  
    text=text.strip()
    #text=text.replace('\N','')
    #text = text.encode("ascii", "ignore")
    #text = text.decode()
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    #text = re.sub(r'\N',' ',text) 
    return text

In [14]:
df3['input_str']=df3['input_str'].progress_apply(lambda x: preprocess(x))

  0%|          | 0/159707 [00:00<?, ?it/s]

In [15]:
df3.to_csv('./output/final.csv')

In [16]:
#def replace_newline(text):
 #   text = r'{}'.format(text)
  #  return text

In [17]:
df3.isnull().sum()

item_name             0
description           0
establishment_type    0
CAT_Name              0
input_str             0
dtype: int64

In [18]:
import torch
from tqdm.notebook import tqdm

#from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
import transformers
from transformers import BertTokenizer
#from transformers.modeling_bert import BertModel, BertForMaskedLM


In [19]:
#pip install torch
!pip install transformers

In [20]:
df3['CAT_Name'].value_counts()

CAT_LIQUOR                                                                                                               31340
CAT_WINE                                                                                                                 17513
CAT_BEER                                                                                                                 12595
CAT_ALCOHOL                                                                                                               8640
CAT_TPP                                                                                                                   7953
                                                                                                                         ...  
CAT_DISPOSABLE_GLOVES,CAT_FIRST_AID_KITS                                                                                     1
TRAIT_FLOUR,CAT_CANDY                                                                                          

In [21]:
possible_labels = df3.CAT_Name.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'CAT_WINE': 0,
 'CAT_LIQUOR': 1,
 'CAT_ALCOHOL': 2,
 'CAT_SPARKLING_WINE': 3,
 'CAT_TEA,CONTAINER_BOTTLED': 4,
 'CAT_SOFT_DRINK,CONTAINER_BOTTLED': 5,
 'CAT_COFFEE,CONTAINER_BOTTLED': 6,
 'CAT_NON_ALCOHOLIC_BEER': 7,
 'CAT_BEER': 8,
 'CAT_CIDER': 9,
 'CAT_JUICE,TRAIT_PCT_100': 10,
 'CAT_TPP': 11,
 'CAT_SUPPLEMENTS': 12,
 'CAT_PREPACKAGED_FOOD_SNACK_CHIPS': 13,
 'CAT_CANDY,TRAIT_FLOUR': 14,
 'CAT_PREPACKAGED_FOOD_EGGS': 15,
 'CAT_MILK_COCOA,CONTAINER_BOTTLED': 16,
 'CAT_TPP_SHAMPOOS': 17,
 'CAT_TPP_BATH_GELS': 18,
 'CAT_PREPACKAGED_FOOD,CAT_SNACK': 19,
 'CAT_TPP_SKIN_CARE_PRODUCTS': 20,
 'CAT_PREPACKAGED_FOOD_DESSERTS': 21,
 'CAT_TPP_BODY_LOTION': 22,
 'CAT_TPP_CANDLE': 23,
 'CAT_LIP_BALM': 24,
 'CAT_ENERGY_DRINK,CONTAINER_BOTTLED': 25,
 'CAT_PREPARED_FOOD': 26,
 'CAT_CANDY_COATED_NUTS': 27,
 'CAT_PREPACKAGED_FOOD_SNACK_COOKIES': 28,
 'CAT_PREPACKAGED_FOOD_CRACKERS,CAT_PREPACKAGED_FOOD_SNACK_COOKIES': 29,
 'CAT_PREPACKAGED_FOOD': 30,
 'CAT_PREPACKAGED_FOOD,CAT_ICECREAM': 31,
 'CAT_PREP

In [22]:
df3['label'] = df3.CAT_Name.replace(label_dict)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df3.input_str).toarray()
labels = df3.label
features.shape

(159707, 72512)

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(df3['input_str'], df3['CAT_Name'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [49]:
print(clf.predict(count_vect.transform(["legent two true legends kentucky straight bour.."])))

['CAT_LIQUOR']


In [53]:
y_pred = clf.predict(count_vect.transform(X_test))
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.6164249755804343


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                                                                                                                                                                                   precision    recall  f1-score   support

                                                                                                                                                                                                                      CAT_ALCOHOL       0.45      0.32      0.37      2131
                                                                                                                                                                                                             CAT_ALCOHOL,CAT_BEER       0.00      0.00      0.00       102
                                                                                                                                                                                CAT_ALCOHOL,CAT_BEER,C

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
#my_tags =df3['']
len(possible_labels)
len(y_pred)
X_test


31347     casamigos añejo ml abv mexico soft caramel and...
122616                        rock candy orange the grocery
87314     lagunitas hazy memory ipa pack oz cans we re g...
67999     hadalabo shirojun premium facial serum ml grocery
96932     menudo menudo comes with pico de gallo and two...
                                ...                        
93171     macallan double cask yer the macallan double c...
49791           dos equis lager pkb oz beer abv the grocery
57966                     fitvine chardonnay ml the grocery
107954    ocean bomb orange dbz limited edition ml taiwa...
89243     legent two true legends kentucky straight bour...
Name: input_str, Length: 39927, dtype: object

In [45]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.metrics import classification_report
y_pred = clf.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

ValueError: could not convert string to float: 'casamigos añejo ml abv mexico soft caramel and vanilla notes with a sweetness from the blue weber agave layered with barrel oak and subtle hints of spice aged for months gives this outstanding tequila a lingering smooth finish enjoy grocery'

In [46]:
y_pred = clf.predict(X_test[100])

ValueError: Expected 2D array, got scalar array instead:
array=patrón silver tequila ml bottle the grocery.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [47]:
X_test[100]

'patrón silver tequila ml bottle the grocery'

In [1]:
#df3[df3['input_str'] == "tito s handmade vodka abv available ice cold ... "]
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

NameError: name 'pd' is not defined

In [ ]:
model = LinearSVC()
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=category_id_df.Product.values, yticklabels=category_id_df.Product.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
df3['label'] = df3.CAT_Name.replace(label_dict)
#stratify=df3.label.values

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df3.index.values, 
                                                  df3.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42 
                                                  )

df3['data_type'] = ['not_set']*df3.shape[0]

df3.loc[X_train, 'data_type'] = 'train'
df3.loc[X_val, 'data_type'] = 'val'

df3.groupby(['CAT_Name', 'label', 'data_type']).count()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df3[df3.data_type=='train'].input_str.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df3[df3.data_type=='val'].input_str.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df3[df3.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df3[df3.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
#device = torch.device()
model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)